In [1]:
%pip install contractions
%pip install textblob
%pip install nltk
%pip install scikit-learn
%pip install transformers
%pip install emoji
%pip install spacy
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.3/624.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 95.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import pandas as pd
import re
from sklearn.feature_extraction import text
import string
import contractions
import emoji
import spacy
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Loading the data:

In [4]:
data = pd.read_csv("filtered_data.csv")
data.head()

,post_id,subreddit,post_title,post_body,number_of_comments,readable_datetime,post_author,number_of_upvotes,query,text,comment_id,comment_body,comment_author
0,1002dom,technology,"ChatGPT Caused 'Code Red' at Google, Report Says",NaN,370,2023-01-01 00:03:33,slakmehl,792,ChatGPT,"ChatGPT Caused 'Code Red' at Google, Report Sa...",NaN,NaN,NaN
1,101o6zx,singularity,"how could someone use ChatGPT or other ""AI"" se...",I'm just looking to make some extra cash on th...,6,2023-01-03 01:25:17,theferalturtle,4,gpt OR GPT or Gpt,"how could someone use ChatGPT or other ""AI"" se...",NaN,NaN,NaN
2,101p00n,OpenAI,Chat gpt won’t verify my phone number,Hello guys when i try to log in to chat gpt it...,73,2023-01-03 01:56:45,T-boner970,3,coding with ChatGPT,Chat gpt won’t verify my phone number Hello gu...,NaN,NaN,NaN
3,102ci8x,OpenAI,Is it possible to use Whisper function for zoo...,"Hello, I am a recent Mathematics undergraduate...",11,2023-01-03 20:34:00,LoanOne2968,1,Whisper,Is it possible to use Whisper function for zoo...,NaN,NaN,NaN
4,102jcse,OpenAI,How could ChatGPT replace Google Search for ma...,NaN,9,2023-01-04 01:03:24,keyhell,1,ChatGPT search,How could ChatGPT replace Google Search for ma...,NaN,NaN,NaN


Converting texts to lowercase:

In [5]:
def lower(text):
  return text.lower()

data["Cleaned Text"] = data["text"].apply(lower)
data["Cleaned Text"].head()

,Cleaned Text
0,"chatgpt caused 'code red' at google, report sa..."
1,"how could someone use chatgpt or other ""ai"" se..."
2,chat gpt won’t verify my phone number hello gu...
3,is it possible to use whisper function for zoo...
4,how could chatgpt replace google search for ma...


Removing r/, usernames, new line indicators, and links from texts:

In [6]:
def remove_links(text):
  return re.sub(r'http[s]?://\S+|www\.\S+', '', text)

def remove_user_mentions(text):
    return re.sub(r'u/\S+', '', text)

data["Cleaned Text"] = data["Cleaned Text"].str.rstrip(" none")
data["Cleaned Text"] = data["Cleaned Text"].str.replace('r/', '', regex=False)
data["Cleaned Text"] = data["Cleaned Text"].str.replace("\n\n", ' ', regex=False)
data["Cleaned Text"] = data["Cleaned Text"].apply(remove_links)
data["Cleaned Text"] = data["Cleaned Text"].apply(remove_user_mentions)

data["Cleaned Text"].head()

,Cleaned Text
0,"chatgpt caused 'code red' at google, report says"
1,"how could someone use chatgpt or other ""ai"" se..."
2,chat gpt won’t verify my phone number hello gu...
3,is it possible to use whisper function for zoo...
4,how could chatgpt replace google search for ma...


Expanding contractions:

In [7]:
def expand_contractions(text):
    return contractions.fix(text)

data["Cleaned Text"] = data["Cleaned Text"].apply(expand_contractions)
data["Cleaned Text"].head()

,Cleaned Text
0,"chatgpt caused 'code red' at google, report says"
1,"how could someone use chatgpt or other ""ai"" se..."
2,chat gpt will not verify my phone number hello...
3,is it possible to use whisper function for zoo...
4,how could chatgpt replace google search for ma...


Converting emojis to their descriptive names:

In [8]:
def convert_emojis(text):
   return emoji.demojize(text)

data["Cleaned Text"] = data["Cleaned Text"].apply(convert_emojis)
data["Cleaned Text"].head()

,Cleaned Text
0,"chatgpt caused 'code red' at google, report says"
1,"how could someone use chatgpt or other ""ai"" se..."
2,chat gpt will not verify my phone number hello...
3,is it possible to use whisper function for zoo...
4,how could chatgpt replace google search for ma...


Removing non-ASCII characters:

In [9]:
def remove_non_ascii(text):
  return text.encode("ascii", "ignore").decode()

data["Cleaned Text"] = data["Cleaned Text"].apply(remove_non_ascii)
data["Cleaned Text"].head()

,Cleaned Text
0,"chatgpt caused 'code red' at google, report says"
1,"how could someone use chatgpt or other ""ai"" se..."
2,chat gpt will not verify my phone number hello...
3,is it possible to use whisper function for zoo...
4,how could chatgpt replace google search for ma...


Removing punctuations:

In [10]:
def remove_punctuations(text):
    text = re.sub(r'[-]', ' ', text)
    text = re.sub(r'(\S)[' + re.escape(string.punctuation) + r'](\S)', r'\1 \2', text)
    return text

data["Cleaned Text"] = data["Cleaned Text"].apply(remove_punctuations)

pattern_punctuations = r'[' + string.punctuation + r']'

data["Cleaned Text"] = data["Cleaned Text"].str.replace(pattern_punctuations, '', regex=True)

data["Cleaned Text"].head()

,Cleaned Text
0,chatgpt caused code red at google report says
1,how could someone use chatgpt or other ai serv...
2,chat gpt will not verify my phone number hello...
3,is it possible to use whisper function for zoo...
4,how could chatgpt replace google search for ma...


Removing numbers:

In [11]:
def remove_numbers(text):
    return re.sub(r'[0-9]+', '', text)

data["Cleaned Text"] = data["Cleaned Text"].apply(remove_numbers)
data['Cleaned Text'].head()

,Cleaned Text
0,chatgpt caused code red at google report says
1,how could someone use chatgpt or other ai serv...
2,chat gpt will not verify my phone number hello...
3,is it possible to use whisper function for zoo...
4,how could chatgpt replace google search for ma...


Correcting words not related to ChatGPT:

In [12]:
protected_words = ["chatgpt", "openai", "gpt", "ai", "llm", "agi", "claude", "anthropic", "google", "deepmind", "bard", "gemini",
    "perplexity", "bing", "copilot", "huggingface", "tensorflow", "pytorch", "machine learning", "neural network", "reinforcement learning",
    "fine tuning", "prompt engineering", "hallucinations", "context window", "token limit", "api", "open source", "llmops", "nlp", "transformer",
    "self attention", "meta ai", "stability ai", "autogpt", "babyagi", "langchain", "generative ai", "multi-modal ai", "ethical ai", "ai bias",
    "parameters", "compute", "training data", "inference", "zero shot", "few shot", "microsoft ai", "amazon bedrock", "cohere", "ai21 labs", "inflection ai",
    "open-source llm", "eleuther ai", "mosaic ml", "cerebras", "hugging face", "stability ai", "databricks", "nvidia nemo", "anthropic ai", "meta llm",
    "system prompt", "temperature", "top k", "top p", "function calling", "retrieval augmented generation", "rag", "hallucination", "ai generated",
    "llm assistant", "ai chatbot", "fine tuned model", "guardrails", "moderation", "chatbot", "embeddings", "rlhf", "supervised learning",
    "self supervised learning", "deep learning", "activation function", "scaling laws", "pre training", "tokenization", "prompt engineering", "prompt injection",
    "adversarial prompting", "jailbreak", "role prompting", "chain-of-thought", "few-shot prompting", "zero-shot prompting", "cot", "cot prompting", "scratchpad",
    "multi-turn conversation", "nucleus sampling", "context window", "token limit", "inference", "parameters", "compute", "retrieval augmented generation",
    "rag", "knowledge distillation", "function calling", "system prompt", "instruction tuning", "fine-tuned model", "chatbot", "ai-generated content",
    "ethical ai", "ai bias", "hallucination", "model drift", "ai explainability", "interpretability", "guardrails", "moderation", "alignment", "rlhf", "constitutional ai",
    "copilot", "huggingface", "stability ai", "meta ai", "mistral ai", "command r", "mixtral", "pythia", "falcon", "llama", "llama-2", "llama-3"]

def correct_spelling(text):
    words = text.split()
    corrected_words = []

    for word in words:
        if word in protected_words:
            corrected_words.append(word)  # Keep the word as is
        else:
            corrected_words.append(str(TextBlob(word).correct()))  # Correct other words

    return " ".join(corrected_words)

data["Cleaned Text"] = data["Cleaned Text"].apply(correct_spelling)
data["Cleaned Text"].head()

### TF-IDF Vectorizer

In [13]:
nlp = spacy.load("en_core_web_sm")

def custom_tokenizer(text):
    doc = nlp(text)
    tokens = []
    for ent in doc.ents:
        tokens.append(ent.text)

    non_entity_tokens = [token.lemma_.lower() for token in doc if not token.ent_type_ and not token.is_punct and not token.is_space]
    tokens.extend(non_entity_tokens)
    return tokens

vectorizer = TfidfVectorizer(
    tokenizer=custom_tokenizer,
    use_idf=False,
    stop_words="english",
    min_df=5
)

doc_vectors = vectorizer.fit_transform(data["Cleaned Text"]).astype("float32")
feature_names = vectorizer.get_feature_names_out()
tfidf_data = pd.DataFrame.sparse.from_spmatrix(doc_vectors, columns=feature_names)
print(tfidf_data.head())

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['far', 'make', 'whereaft'] not in stop_words.
  warnings.warn(


   \  a billion dollars  a couple days  a couple months  a couple of days  \
0  0                  0              0                0                 0   
1  0                  0              0                0                 0   
2  0                  0              0                0                 0   
3  0                  0              0                0                 0   
4  0                  0              0                0                 0   

   a couple of hours  a couple of weeks  a couple of years  a couple weeks  \
0                  0                  0                  0               0   
1                  0                  0                  0               0   
2                  0                  0                  0               0   
3                  0                  0                  0               0   
4                  0                  0                  0               0   

   a couple weeks ago  ...  young  youth  youtuber  z  zero  zerogpt

In [14]:
query = ["chatgpt", "performance", "user experiences", "applications", "limitations", "ethical concerns", "comparisons", "ai model", "openai", "deepseek", "claude",
         "chatbot", "gpt", "language model", "ai assistant", "nlp", "machine learning", "deep learning", "ai-generated content", "ai reasoning", "automated responses",
         "performance", "pricing", "cost", "privacy", "quality", "rate limit", "review", "security", "sora", "speed", "subscription", "chatgpt plus", "benchmarks",
         "quality", "latency", "llama"]

idf_vectorizer = TfidfVectorizer(
    tokenizer=custom_tokenizer,
    use_idf=True,
    # turn off sublinear_tf to get the same results as the previous implementation
    sublinear_tf=False,
    stop_words="english",
    # using the vocabulary from the matrix of the document because min_df=5 cannot be used here (it will ignore words appearing less than 5 times in the query too)
    vocabulary=vectorizer.vocabulary_
)

query_vector = idf_vectorizer.fit_transform(query)

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['far', 'make', 'whereaft'] not in stop_words.
  warnings.warn(


In [15]:
similarity_scores = cosine_similarity(query_vector, doc_vectors)[0]

data["similarity"] = similarity_scores

sorted_data = data.sort_values(by="similarity", ascending=False)

sorted_data.to_csv("similarity_scores.csv", index=False)

print("Data saved to 'similarity_scores.csv'")

Data saved to 'similarity_scores.csv'


In [16]:
sorted_data.head()

,post_id,subreddit,post_title,post_body,number_of_comments,readable_datetime,post_author,number_of_upvotes,query,text,comment_id,comment_body,comment_author,Cleaned Text,similarity
5851,1id38mh,ChatGPT,Chatgpt is down?,NaN,15,2025-01-30 00:08:03,tivel8571,7,NaN,Chatgpt is down? None,NaN,NaN,NaN,chatgpt is down,1.0
30273,1f477p6,OpenAI,For teachers - Claude vs Chat GPT?,I am an online English teacher. Any teachers t...,17,2024-08-29 20:46:21,Parking_Rub_1701,1,ChatGPT vs Claude,ChatGPT with GPTs,lkle1tz,ChatGPT with GPTs,OtherwiseLiving,chatgpt with gets,1.0
41680,1hmv4v8,OpenAI,CHAT GPT IS DOWN.,NaN,380,2024-12-26 22:55:29,Crystaleana,25,availability OR downtime OR outage,https://preview.redd.it/hn2eap1u299e1.jpeg?wid...,m3x9pj6,https://preview.redd.it/hn2eap1u299e1.jpeg?wid...,SnooComics7329,not only chatgpt,1.0
5671,1icmp32,ChatGPT,Not much better than chatgpt,NaN,3,2025-01-29 09:26:17,IotaNine,1,NaN,Not much better than chatgpt None,NaN,NaN,NaN,not much better than chatgpt,1.0
2712,1d82ug3,OpenAI,ChatGPT is up again?,NaN,34,2024-06-04 21:27:29,No-Conference-8133,17,ChatGPT vs Gemini,ChatGPT is up again? None,NaN,NaN,NaN,chatgpt is up again,1.0
